In [27]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [28]:
# Load the Excel file
file_name = "SES_2024_tidy.xlsx.coredownload.xlsx"
sheet_name = "T3.5"

# Read the data from the Excel file
df = pd.read_excel(file_name, sheet_name=sheet_name)
df = df.fillna(0)

# Treat the random 's' as 0
df['Kwh_Per_Acc_Cleaned'] = df['Kwh Per Acc'].apply(lambda x: 0 if isinstance(x, str) and 's' in x else x) 

# Filter the data for the year 2024 and 'Dwelling Type' as 'overall'
filtered_data = df[(df['Year'] == 2024) & (df['Dwelling Type'] == 'Overall')]

# Calculate total consumption per region
regions = ['Central Region', 'East Region', 'North East Region', 'North Region', 'West Region']
demand_kW_per_region = {
    region: filtered_data[filtered_data['Region'] == region]['Kwh_Per_Acc_Cleaned'].astype(float).sum()
    for region in regions
}

demand_kW_per_region

{'Central Region': 88877.8,
 'East Region': 27358.199999999997,
 'North East Region': 23995.5,
 'North Region': 23482.0,
 'West Region': 19945.699999999997}

In [29]:
# Define other parameters
types_of_panels = ['Monocrystalline', 'Polycrystalline', 'Thin-film']
efficiency = {'Monocrystalline': 0.15, 'Polycrystalline': 0.13, 'Thin-film': 0.07}
fixed_cost = {'Monocrystalline': 0.75, 'Polycrystalline': 0.70, 'Thin-film': 0.30}
install_cost = {'Monocrystalline': 300, 'Polycrystalline': 250, 'Thin-film': 200}
maintenance_cost_percentage = 0.01
panel_area_ft2 = 15
panel_area_m2 = panel_area_ft2 * 0.092903
available_space_m2 = 2559.5

In [30]:
# Create a new model
model = gp.Model("solar_optimization")

# Decision variables
x = model.addVars(regions, types_of_panels, vtype=GRB.INTEGER, name="x")
y = model.addVars(regions, types_of_panels, vtype=GRB.BINARY, name="y")

In [31]:
# Objective function: Minimize total cost
model.setObjective(
    gp.quicksum(
        (fixed_cost[j] + install_cost[j] + maintenance_cost_percentage * (fixed_cost[j] + install_cost[j])) * x[i, j]
        for i in regions for j in types_of_panels
    ),
    GRB.MINIMIZE
)

In [32]:
# Constraints
for i in regions:
    # Demand constraint
    model.addConstr(
        gp.quicksum(efficiency[j] * x[i, j] * panel_area_m2 for j in types_of_panels) >= demand_kW_per_region[i],
        name=f"demand_{i}"
    )

    # Space constraint
    model.addConstr(
        gp.quicksum(x[i, j] * panel_area_m2 for j in types_of_panels) <= available_space_m2,
        name=f"space_{i}"
    )

    # Installation constraints
    for j in types_of_panels:
        model.addConstr(x[i, j] <= 1000 * y[i, j], name=f"install_{i}_{j}")

In [33]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5825U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 25 rows, 30 columns and 60 nonzeros
Model fingerprint: 0xfd736daf
Variable types: 0 continuous, 30 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [2e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 9e+04]
Presolve removed 15 rows and 15 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [34]:
# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for i in regions:
        for j in types_of_panels:
            print(f"Region: {i}, {j}: {x[i, j].x} panels, Build: {y[i, j].x}")
    print(f"Total Cost: ${model.objVal}")
else:
    print("No optimal solution found.")

No optimal solution found.


In [35]:
for constr in model.getConstrs():
    print(f"Constraint: {constr.ConstrName}, Expression: {model.getRow(constr)}, RHS: {constr.RHS}")

Constraint: demand_Central Region, Expression: 0.20903175 x[Central Region,Monocrystalline] + 0.18116085 x[Central Region,Polycrystalline] + 0.09754815000000001 x[Central Region,Thin-film], RHS: 88877.8
Constraint: space_Central Region, Expression: 1.393545 x[Central Region,Monocrystalline] + 1.393545 x[Central Region,Polycrystalline] + 1.393545 x[Central Region,Thin-film], RHS: 2559.5
Constraint: install_Central Region_Monocrystalline, Expression: x[Central Region,Monocrystalline] + -1000.0 y[Central Region,Monocrystalline], RHS: 0.0
Constraint: install_Central Region_Polycrystalline, Expression: x[Central Region,Polycrystalline] + -1000.0 y[Central Region,Polycrystalline], RHS: 0.0
Constraint: install_Central Region_Thin-film, Expression: x[Central Region,Thin-film] + -1000.0 y[Central Region,Thin-film], RHS: 0.0
Constraint: demand_East Region, Expression: 0.20903175 x[East Region,Monocrystalline] + 0.18116085 x[East Region,Polycrystalline] + 0.09754815000000001 x[East Region,Thin-fi